In [1]:
from webscraping import *

c:\Users\Adm\Desktop\Rafael\Nova pasta\Base_flashscore\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')               # Executa em modo headless (sem interface gráfica)
    options.add_argument('--disable-gpu')            # Melhor performance
    options.add_argument('--no-sandbox')             # Necessário para execução em alguns servidores
    options.add_argument('--disable-dev-shm-usage')  # Evita problemas de memória
    service = Service('./chromedriver.exe')          # Caminho para o chromedriver
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window()
    return driver

In [3]:
def process_url(url, colecao_partidas):
    """
    Processa uma URL para extrair dados dos jogos e armazená-los no banco de dados.
    """
    driver = setup_driver()
    try:
        driver.get(url)
        if 'No match found.' in driver.page_source:
            logging.info(f"Nenhuma partida encontrada na URL: {url}")
            return

        # Aceitar cookies (se necessário)
        try:
            WebDriverWait(driver, 8).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler'))
            )
            driver.find_element(By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler').click()
        except TimeoutException:
            logging.info("Cookies já foram aceitos ou não encontrados.")

        # Carregar mais eventos, se disponíveis
        while 'event__more' in driver.page_source:
            try:
                button_more = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, 'a.event__more.event__more--static')
                    )
                )
                driver.execute_script("arguments[0].click();", button_more)
                sleep(2)
            except TimeoutException:
                logging.error("Timeout ao tentar carregar mais eventos.")
                break

        # Extrair IDs dos jogos
        jogos = driver.find_elements(By.CSS_SELECTOR, 'div.event__match--twoLine')
        id_jogos = [jogo.get_attribute("id")[4:] for jogo in jogos if jogo.get_attribute("id")]

        # Extrair temporada
        try:
            season = driver.find_element(By.CSS_SELECTOR, 'div.heading__info').text
        except Exception as e:
            logging.error(f"Erro ao extrair temporada: {e}")
            season = "Unknown"

        logging.info(f"Processando temporada: {season}")

        # Processar cada jogo encontrado
        for id_jogo in tqdm(id_jogos, desc=f"Processando {driver.title}"):
            query = Query()
            if not colecao_partidas.search(query.Id == id_jogo):
                try:
                    jogo = {}
                    Dados_Jogo(id_jogo, jogo, driver) 
                    Temporada(jogo, season)  
                    Odds_Math_Odds_HT(id_jogo, jogo, driver)
                    Odds_Over_Under_HT(id_jogo, jogo, driver)
                    Odds_Math_Odds_FT(id_jogo, jogo, driver)
                    Odds_Over_Under_FT(id_jogo, jogo, driver)
                    Odds_BTTS(id_jogo, jogo, driver)
                    Odds_Dupla_Chance(id_jogo, jogo, driver)
                    Goals_HT_FT(id_jogo, jogo, driver)
                    Minutos_dos_Gols(id_jogo, jogo, driver)

                    colecao_partidas.insert(jogo.copy())
                    logging.info(f"Jogo {id_jogo} inserido com sucesso.")
                except Exception as e:
                    logging.error(f"Erro ao processar jogo {id_jogo}: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar URL {url}: {e}")
    finally:
        driver.quit()

In [4]:
links_2025 = [
    'https://www.flashscore.com/football/argentina/torneo-betano-2025/results/',
    'https://www.flashscore.com/football/argentina/copa-de-la-liga-profesional-2025/results/',
    'https://www.flashscore.com/football/bolivia/division-profesional-2025/results/',
    'https://www.flashscore.com/football/bolivia/nacional-b-2025/results/',
    'https://www.flashscore.com/football/brazil/serie-a-betano-2025/results/',
    'https://www.flashscore.com/football/brazil/serie-b-2025/results/',
    'https://www.flashscore.com/football/chile/liga-de-primera-2025/results/',
    'https://www.flashscore.com/football/chile/liga-de-ascenso-2025/results/',
    'https://www.flashscore.com/football/china/super-league-2025/results/',
    'https://www.flashscore.com/football/china/jia-league-2025/results/',
    'https://www.flashscore.com/football/colombia/primera-a-2025/results/',
    'https://www.flashscore.com/football/colombia/primera-b-2025/results/',
    'https://www.flashscore.com/football/ecuador/liga-pro-2025/results/',
    'https://www.flashscore.com/football/ecuador/serie-b-2025/results/',
    'https://www.flashscore.com/football/estonia/meistriliiga-2025/results/',
    'https://www.flashscore.com/football/estonia/esiliiga-2025/results/',
    'https://www.flashscore.com/football/finland/veikkausliiga-2025/results/',
    'https://www.flashscore.com/football/finland/ykkosliiga-2025/results/',
    'https://www.flashscore.com/football/iceland/besta-deild-karla-2025/results/',
    'https://www.flashscore.com/football/iceland/division-1-2025/results/',
    'https://www.flashscore.com/football/ireland/premier-division-2025/results/',
    'https://www.flashscore.com/football/ireland/division-1-2025/results/',
    'https://www.flashscore.com/football/japan/j1-league-2025/results/',
    'https://www.flashscore.com/football/japan/j2-league-2025/results/',
    'https://www.flashscore.com/football/norway/eliteserien-2025/results/',
    'https://www.flashscore.com/football/norway/obos-ligaen-2025/results/',
    'https://www.flashscore.com/football/paraguay/copa-de-primera-2025/results/',
    'https://www.flashscore.com/football/paraguay/division-intermedia-2025/results/',
    'https://www.flashscore.com/football/peru/liga-1-2025/results/',
    'https://www.flashscore.com/football/peru/liga-2-2025/results/',
    'https://www.flashscore.com/football/south-america/copa-libertadores-2025/results/',
    'https://www.flashscore.com/football/south-america/copa-sudamericana-2025/results/',
    'https://www.flashscore.com/football/south-korea/k-league-1-2025/results/',
    'https://www.flashscore.com/football/south-korea/k-league-2-2025/results/',
    'https://www.flashscore.com/football/sweden/allsvenskan-2025/results/',
    'https://www.flashscore.com/football/sweden/superettan-2025/results/',
    'https://www.flashscore.com/football/uruguay/liga-auf-uruguaya-2025/results/',
    'https://www.flashscore.com/football/uruguay/segunda-division-2025/results/',
    'https://www.flashscore.com/football/usa/mls-2025/results/',
    'https://www.flashscore.com/football/usa/usl-championship-2025/results/',
    'https://www.flashscore.com/football/venezuela/liga-futve-2025/results/',
    'https://www.flashscore.com/football/venezuela/liga-futve-2-2025/results/',
]

links_2024_2025 = [
    'https://www.flashscore.com/football/australia/a-league-2024-2025/results/',
    'https://www.flashscore.com/football/austria/bundesliga-2024-2025/results/',
    'https://www.flashscore.com/football/austria/2-liga-2024-2025/results/',
    'https://www.flashscore.com/football/belgium/jupiler-pro-league-2024-2025/results/',
    'https://www.flashscore.com/football/belgium/challenger-pro-league-2024-2025/results/',
    'https://www.flashscore.com/football/bosnia-and-herzegovina/wwin-liga-bih-2024-2025/results/',
    'https://www.flashscore.com/football/bulgaria/parva-liga-2024-2025/results/',
    'https://www.flashscore.com/football/croatia/hnl-2024-2025/results/',
    'https://www.flashscore.com/football/croatia/prva-nl-2024-2025/results/',
    'https://www.flashscore.com/football/cyprus/cyprus-league-2024-2025/results/',
    'https://www.flashscore.com/football/czech-republic/chance-liga-2024-2025/results/',
    'https://www.flashscore.com/football/denmark/superliga-2024-2025/results/',
    'https://www.flashscore.com/football/denmark/1st-division-2024-2025/results/',
    'https://www.flashscore.com/football/egypt/premier-league-2024-2025/results/',
    'https://www.flashscore.com/football/england/premier-league-2024-2025/results/',
    'https://www.flashscore.com/football/england/championship-2024-2025/results/',
    'https://www.flashscore.com/football/england/league-one-2024-2025/results/',
    'https://www.flashscore.com/football/england/league-two-2024-2025/results/',
    'https://www.flashscore.com/football/england/national-league-2024-2025/results/',
    'https://www.flashscore.com/football/europe/champions-league-2024-2025/results/',
    'https://www.flashscore.com/football/europe/europa-league-2024-2025/results/',
    'https://www.flashscore.com/football/europe/europa-conference-league-2024-2025/results/',
    'https://www.flashscore.com/football/france/ligue-1-2024-2025/results/',
    'https://www.flashscore.com/football/france/ligue-2-2024-2025/results/',
    'https://www.flashscore.com/football/germany/bundesliga-2024-2025/results/',
    'https://www.flashscore.com/football/germany/2-bundesliga-2024-2025/results/',
    'https://www.flashscore.com/football/greece/super-league-2024-2025/results/',
    'https://www.flashscore.com/football/greece/super-league-2-2024-2025/results/',
    'https://www.flashscore.com/football/israel/ligat-ha-al-2024-2025/results/',
    'https://www.flashscore.com/football/italy/serie-a-2024-2025/results/',
    'https://www.flashscore.com/football/italy/serie-b-2024-2025/results/',
    'https://www.flashscore.com/football/mexico/liga-mx-2024-2025/results/',
    'https://www.flashscore.com/football/mexico/liga-de-expansion-mx-2024-2025/results/',
    'https://www.flashscore.com/football/netherlands/eredivisie-2024-2025/results/',
    'https://www.flashscore.com/football/netherlands/eerste-divisie-2024-2025/results/',
    'https://www.flashscore.com/football/northern-ireland/nifl-premiership-2024-2025/results/',
    'https://www.flashscore.com/football/northern-ireland/nifl-championship-2024-2025/results/',
    'https://www.flashscore.com/football/poland/ekstraklasa-2024-2025/results/',
    'https://www.flashscore.com/football/poland/division-1-2024-2025/results/',
    'https://www.flashscore.com/football/portugal/liga-portugal-2024-2025/results/',
    'https://www.flashscore.com/football/portugal/liga-portugal-2-2024-2025/results/',
    'https://www.flashscore.com/football/romania/superliga-2024-2025/results/',
    'https://www.flashscore.com/football/romania/liga-2-2024-2025/results/',
    'https://www.flashscore.com/football/saudi-arabia/saudi-professional-league-2024-2025/results/',
    'https://www.flashscore.com/football/scotland/premiership-2024-2025/results/',
    'https://www.flashscore.com/football/scotland/championship-2024-2025/results/',
    'https://www.flashscore.com/football/serbia/super-liga-2024-2025/results/',
    'https://www.flashscore.com/football/serbia/prva-liga-2024-2025/results/',
    'https://www.flashscore.com/football/slovakia/nike-liga-2024-2025/results/',
    'https://www.flashscore.com/football/slovakia/2-liga-2024-2025/results/',
    'https://www.flashscore.com/football/spain/laliga-2024-2025/results/',
    'https://www.flashscore.com/football/spain/laliga2-2024-2025/results/',
    'https://www.flashscore.com/football/switzerland/super-league-2024-2025/results/',
    'https://www.flashscore.com/football/switzerland/challenge-league-2024-2025/results/',
    'https://www.flashscore.com/football/turkey/super-lig-2024-2025/results/',
    'https://www.flashscore.com/football/turkey/1-lig-2024-2025/results/',
    'https://www.flashscore.com/football/ukraine/premier-league-2024-2025/results/',
    'https://www.flashscore.com/football/wales/cymru-premier-2024-2025/results/'
]

In [5]:
db = TinyDB('./base_flashscore_2025.json')
for url in links_2025:
    logging.info(f"Iniciando processamento para: {url}")
    process_url(url, db)
logging.info("Processamento concluído.")

Processando Torneo Betano 2025 Results - Football/Argentina: 100%|██████████| 119/119 [00:01<00:00, 75.02it/s]
Processando Chilean Primera División Results - Football/Chile: 100%|██████████| 24/24 [00:20<00:00,  1.16it/s]
Processando Liga de Ascenso 2025 Results - Football/Chile: 100%|██████████| 16/16 [00:00<00:00, 65.54it/s]
Processando Super League 2025 Results - Football/China: 100%|██████████| 16/16 [00:00<00:00, 57.60it/s]
Processando Primera A 2025 Results - Football/Colombia: 100%|██████████| 66/66 [00:04<00:00, 13.27it/s]
Processando Primera B 2025 Results - Football/Colombia: 100%|██████████| 38/38 [00:01<00:00, 35.95it/s]
Processando Liga Pro 2025 Results - Football/Ecuador: 100%|██████████| 24/24 [00:00<00:00, 39.07it/s]
Processando Meistriliiga 2025 Results - Football/Estonia: 100%|██████████| 5/5 [00:00<00:00, 57.54it/s]
Processando Esiliiga 2025 Results - Football/Estonia: 100%|██████████| 6/6 [00:08<00:00,  1.37s/it]
Processando Irish Premier Division Results - Football

In [ ]:
db = TinyDB('./base_flashscore_2024_2025.json')
for url in links_2024_2025:
    logging.info(f"Iniciando processamento para: {url}")
    process_url(url, db)
logging.info("Processamento concluído.")

Processando A-League 2025 Results - Football/Australia: 100%|██████████| 121/121 [00:48<00:00,  2.52it/s]
Processando Bundesliga 2025 Results - Football/Austria: 100%|██████████| 120/120 [00:08<00:00, 14.52it/s]
Processando 2. Liga 2025 Results - Football/Austria: 100%|██████████| 144/144 [00:08<00:00, 16.26it/s]
Processando Jupiler Pro League 2025 Results - Football/Belgium: 100%|██████████| 224/224 [00:11<00:00, 20.06it/s]
Processando Challenger Pro League 2025 Results - Football/Belgium: 100%|██████████| 182/182 [00:09<00:00, 18.46it/s]
Processando Premijer liga Results - Football/Bosnia and Herzegovina:  44%|████▎     | 52/119 [00:01<00:02, 32.73it/s]2025-03-07 08:07:56,182 - ERROR - Erro ao processar jogo lleCvyal: list index out of range
Processando Premijer liga Results - Football/Bosnia and Herzegovina: 100%|██████████| 119/119 [00:18<00:00,  6.41it/s]
Processando Parva liga 2025 Results - Football/Bulgaria: 100%|██████████| 184/184 [00:10<00:00, 17.30it/s]
Processando HNL 2025

Jogo AkCmzyWI não se realizou. Ignorando HT e ST.


Processando Cyprus League 2025 Results - Football/Cyprus: 100%|██████████| 175/175 [43:17<00:00, 14.84s/it]
Processando Chance Liga 2025 Results - Football/Czech Republic:  31%|███       | 58/190 [14:28<32:15, 14.66s/it]

Erro geral ao processar o jogo jJdP8tYO: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A253DF85+26693]
	(No symbol) [0x00007FF7A249EAD0]
	(No symbol) [0x00007FF7A23291CA]
	(No symbol) [0x00007FF7A237F747]
	(No symbol) [0x00007FF7A237F97C]
	(No symbol) [0x00007FF7A23D33C7]
	(No symbol) [0x00007FF7A23A792F]
	(No symbol) [0x00007FF7A23D00B8]
	(No symbol) [0x00007FF7A23A76C3]
	(No symbol) [0x00007FF7A2370490]
	(No symbol) [0x00007FF7A2371743]
	GetHandleVerifier [0x00007FF7A289436D+3525677]
	GetHandleVerifier [0x00007FF7A28A7F3B+3606523]
	GetHandleVerifier [0x00007FF7A289CEE3+3561379]
	GetHandleVerifier [0x00007FF7A2607C0A+853194]
	(No symbol) [0x00007FF7A24A990F]
	(No symbol) [0x00007FF7A24A5674]
	(No symbol) [0x00007FF7A24A5816]
	(No symbol) [0x00007FF7A2494D89]
	BaseThreadInitThunk [0x00007FFA8D777374+20]
	RtlUserThreadStart [0x00007FFA8E1BCC91+33]



Processando Chance Liga 2025 Results - Football/Czech Republic: 100%|██████████| 190/190 [47:48<00:00, 15.10s/it]
Processando Superliga 2025 Results - Football/Denmark: 100%|██████████| 120/120 [30:09<00:00, 15.08s/it]
Processando 1st Division 2025 Results - Football/Denmark: 100%|██████████| 120/120 [30:37<00:00, 15.31s/it]
Processando Egyptian Premier League Results - Football/Egypt:   0%|          | 0/153 [00:00<?, ?it/s]2025-03-07 10:44:23,087 - ERROR - Erro em Dados_Jogo: HTTPConnectionPool(host='localhost', port=63572): Read timed out. (read timeout=120)
2025-03-07 10:46:23,115 - ERROR - Erro ao processar jogo tEbgjfs9: HTTPConnectionPool(host='localhost', port=63572): Read timed out. (read timeout=120)
Processando Egyptian Premier League Results - Football/Egypt:   1%|          | 1/153 [04:00<10:08:19, 240.13s/it]2025-03-07 10:48:23,219 - ERROR - Erro em Dados_Jogo: HTTPConnectionPool(host='localhost', port=63572): Read timed out. (read timeout=120)
2025-03-07 10:50:23,229 - ERR

In [ ]:
base_2025 = TinyDB('base_flashscore_2025.json')
base_2024_2025 = TinyDB('base_flashscore_2024_2025.json')

df1 = pd.DataFrame(base_2025.all())
df2 = pd.DataFrame(base_2024_2025.all())
df = pd.concat([df1, df2], ignore_index=True)
df.drop_duplicates(subset=['Id'], inplace=True)

df[['Dia','Mes','Ano']] = df['Date'].str.split('/',expand=True)
df = df.drop(['Date'], axis=1)
df['Date'] = df['Ano']+'-'+df['Mes']+'-'+ df['Dia']
df = df.drop(columns=['Dia','Mes','Ano'], axis=1)

df = df[['Id','League','Season','Round','Date','Time','Home','Away',
         'Odd_H_HT','Odd_D_HT','Odd_A_HT',
         'Odd_Over05_HT','Odd_Under05_HT','Odd_Over15_HT','Odd_Under15_HT','Odd_Over25_HT','Odd_Under25_HT',
         'Odd_H_FT','Odd_D_FT','Odd_A_FT',
         'Odd_Over05_FT','Odd_Under05_FT','Odd_Over15_FT','Odd_Under15_FT','Odd_Over25_FT','Odd_Under25_FT',
         'Odd_Over35_FT','Odd_Under35_FT','Odd_Over45_FT','Odd_Under45_FT',
         'Odd_BTTS_Yes','Odd_BTTS_No',
         'Odd_1X','Odd_12','Odd_X2',
         'Goals_H_HT','Goals_A_HT','Goals_H_FT','Goals_A_FT','Goals_Minutes_Home','Goals_Minutes_Away']]
df.fillna(0, inplace=True)

df = df.sort_values(by='Date')
df = drop_reset_index(df)

display(df)